# Fuzzing with Constraints

In this chapter, we show how to extend grammars with _constraints_ – conditions that are evaluated while a string is produced, and which have to be satisfied.

**Prerequisites**

* You should have read the [chapter on efficient grammar fuzzing](GrammarFuzzer.ipynb).

## Specifying Functions


\todo{Have a check on which expansions are supported}

In [ ]:
import fuzzingbook_utils

In [ ]:
from Grammars import EXPR_GRAMMAR, is_valid_grammar
from ProbabilisticGrammarFuzzer import opts, exp_opts, exp_string
from GrammarFuzzer import GrammarFuzzer, all_terminals
import copy

In [ ]:
def exp_pre_expansion_function(expansion):
    """Return the specified value generator, or None if unspecified"""
    if exp_opts(expansion) is None:
        return None
    return exp_opts(expansion).get('pre', None)

## Generating Elements

Make things even faster: define symbols right away

In [ ]:
import random

In [ ]:
constrained_expr_grammar = copy.deepcopy(EXPR_GRAMMAR)

constrained_expr_grammar.update(
    {
     "<start>": ["<expr>"],
     "<factor>": [
         "+<factor>",
         "-<factor>",
         "(<expr>)",
         ("<integer>.<integer>", opts(pre=lambda: (random.randint(100, 200), None))),
         ("<integer>", opts(pre=lambda: random.randint(100, 200))),
        ],
    }
)

In [ ]:
class ConstraintGrammarFuzzer(GrammarFuzzer):
    pass

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def expansion_to_children(self, expansion):
        children = super().expansion_to_children(expansion)
        function = exp_pre_expansion_function(expansion)
        if function is None:
            return children
        
        assert callable(function)

        result = function()
        if self.log:
            print(repr(function) + "()", "=", repr(result))
        return self.apply_result(result, children)

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def apply_result(self, result, children):
        if isinstance(result, bool):
            pass
        elif isinstance(result, str):
            children = [(result, [])]
        elif isinstance(result, tuple):
            for index, value in enumerate(result):
                if value is not None:
                    if not isinstance(value, str):
                        value = repr(value)
                    if self.log:
                        print("Replacing", all_terminals(children[index]), "by", value)

                    children[index] = (value, [])
        else:
            if self.log:
                print("Replacing", "".join([all_terminals(c) for c in children]), "by", result)

            children = [(repr(result), [])]

        return children

In [ ]:
evaluating_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar, log=True)
evaluating_fuzzer.fuzz()

## Specifying Constraints

In [ ]:
constrained_expr_grammar = copy.deepcopy(EXPR_GRAMMAR)

constrained_expr_grammar.update(
    {
        "<start>": [("<expr>", opts(post=lambda s: eval(s) > 5))]
    }
)

assert is_valid_grammar(constrained_expr_grammar)

In [ ]:
def exp_post_expansion_function(expansion):
    """Return the specified post-expansion function, or None if unspecified"""
    if exp_opts(expansion) is None:
        return None
    return exp_opts(expansion).get('post', None)

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def eval_function(self, tree, function):
        symbol, children = tree
        # print("Does", all_terminals(tree), "satisfy", repr(function) + "?")

        assert callable(function)

        args = []
        for (symbol, exp) in children:
            if exp != [] and exp is not None:
                symbol_value = all_terminals((symbol, exp))
                args.append(symbol_value)
                
        try:
            result = function(*args)
            if self.log:
                print(repr(function) + repr(tuple(args)), "=", repr(result))

        except Exception as exc:
            if self.log:
                print(repr(function) + repr(tuple(args)) + ": " + repr(exc))
            result = None

        return result

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    # Return True iff all constraints of grammar are satisfied in TREE
    def run_functions(self, tree):
        symbol, children = tree
        if symbol not in self.grammar:
            return True

        applied_expansion = \
            "".join([child_symbol for child_symbol, _ in children])

        result = True
        for expansion in self.grammar[symbol]:
            if exp_string(expansion) == applied_expansion:
                function = exp_post_expansion_function(expansion)
                if function is None:
                    break
                result = self.eval_function(tree, function)
                if result:
                    children = self.apply_result(result, children)
                    break

                return False

        for c in children:
            if not self.run_functions(c):
                return False

        return result

The simplest method to check constraints is to retain only those trees that satisfy them.  Works, but can be very slow.

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def fuzz_tree(self):
        while True:
            tree = super().fuzz_tree()
            if self.run_functions(tree):
                return tree

In [ ]:
constraint_grammar_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
expr = constraint_grammar_fuzzer.fuzz()
expr

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    eval(expr)

In [ ]:
constrained_expr_grammar.update(
    {
     "<start>": ["<expr>"],
     "<factor>": [
         "+<factor>",
         "-<factor>",
         "(<expr>)",
         # ("<integer>.<integer>", opts(constraint="int($$) > 10")),
         ("<integer>.<integer>", opts(post=lambda s1, s2: float(s1 + "." + s2) > 10)),
         ("<integer>", opts(post=lambda n: int(n) > 10))
        ],
    }
)

In [ ]:
from Timer import Timer

In [ ]:
constraint_grammar_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
with Timer() as timer:
    print([constraint_grammar_fuzzer.fuzz() for i in range(10)])

In [ ]:
timer.elapsed_time()

## Solving Constraints on the Go

Make things faster: Check as soon as (sub)tree is complete

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def expand_tree_once(self, tree):
        new_tree = super().expand_tree_once(tree)
        
        (symbol, children) = new_tree
        if all([exp_post_expansion_function(expansion) is None for expansion in self.grammar[symbol]]):
            # No constraints for this symbol
            return new_tree
                
        if self.any_possible_expansions(tree):
            # Still expanding
            return new_tree

        result = self.run_functions(new_tree)
        if result:
            # All constraints satisfied
            children = self.apply_result(result, children)
            new_tree = (symbol, children)
            return new_tree

        # Replace tree by unexpanded symbol and try again
        if self.log:
            print(all_terminals(new_tree), "did not satisfy", symbol, "constraints")
            
        if self.replacement_attempts_counter > 0:
            if self.log:
                print("Trying another expansion")
            self.replacement_attempts_counter -= 1
            return (symbol, None)
        
        if self.log:
            print("Starting from scratch")
        raise RestartExpansionException

In [ ]:
class RestartExpansionException(Exception):
    pass

In [ ]:
class ConstraintGrammarFuzzer(ConstraintGrammarFuzzer):
    def __init__(self, grammar, replacement_attempts=10, **kwargs):
        super().__init__(grammar, **kwargs)
        self.replacement_attempts = replacement_attempts

    def fuzz_tree(self):
        while True:
            self.replacement_attempts_counter = self.replacement_attempts
            try:
                tree = super().fuzz_tree()
                return tree
            except RestartExpansionException:
                continue

In [ ]:
constraint_grammar_fuzzer = ConstraintGrammarFuzzer(constrained_expr_grammar)
with Timer() as timer:
    print([constraint_grammar_fuzzer.fuzz() for i in range(10)])

In [ ]:
timer.elapsed_time()

## Usage Examples

### Matching Tags

In [ ]:
from Grammars import crange

In [ ]:
XML_GRAMMAR = {
    "<start>": ["<xml-tree>"],
    "<xml-tree>": ["<start-tag><xml-content><end-tag>"],
    "<xml-content>": ["Text", "<xml-tree>"],
    "<start-tag>": ["<langle><id><rangle>"],
    "<end-tag>": ["<langle>/<id><rangle>"],
    "<langle>": ["<"],
    "<rangle>": [">"],
    "<id>": ["<letter>", "<id><letter>"],
    "<letter>": crange('a', 'z')
}

assert is_valid_grammar(XML_GRAMMAR)

In [ ]:
xml_fuzzer = GrammarFuzzer(XML_GRAMMAR)
xml_fuzzer.fuzz()

In [ ]:
def random_xml():
    xml_id_fuzzer = GrammarFuzzer(XML_GRAMMAR, start_symbol="<id>")
    id = xml_id_fuzzer.fuzz()
    return ("<" + id + ">", None, "</" + id + ">")

XML_GRAMMAR.update({
    "<xml-tree>": [("<start-tag><xml-content><end-tag>",
                    opts(pre=random_xml)
                   )]
})

In [ ]:
xml_fuzzer = ConstraintGrammarFuzzer(XML_GRAMMAR, log=True, disp=True)
xml_fuzzer.fuzz()

### Defining and Using Identifiers

### Checksums

## All Together

In [ ]:
from ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer, ProbabilisticGrammarCoverageFuzzer

In [ ]:
class BigFatGrammarFuzzer(ProbabilisticGrammarFuzzer, ConstraintGrammarFuzzer):
    pass

In [ ]:
class BigFatGrammarCoverageFuzzer(ProbabilisticGrammarCoverageFuzzer, ConstraintGrammarFuzzer):
    pass

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

1. Implement a syntax that allows people to refer to subtrees – say $1.$2 is the second child of the first symbol.


### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

In [ ]:
ATTR_GRAMMAR = {
 "<clause>": ("<xml-open>Text<xml-close>", opts(constraint=lambda x1, x2: x1.name == x2.name)),
 "<xml-open>": ("<langle><tag><rangle>", opts(producer=lambda: (None, opts(name=random_name()), None))),
 "<xml-close>": ("<langle>/<tag><rangle>", opts(producer=lambda: (None, find_name(), None))),
}

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_